In [ ]:
%pylab inline

from lib.KDTreeEncoding import *

import xgboost as xgb
from lib.XGBHelper import *
from lib.XGBoost_params import *
from lib.score_analysis import *

from lib.logger import logger

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from glob import glob
import pandas as pd
import pickle as pkl
from copy import deepcopy

###  load image filenames

In [ ]:
poverty_dir='/mnt/repos/dataset/'
#use the following line when running on datahub
#poverty_dir = '/datasets/cs255-sp22-a00-public/poverty/'
depth=8   #for KDTree

In [ ]:
## load file list
image_dir=poverty_dir+'/anon_images'
files=[]
files+=glob(f'{image_dir}/*.npz')
len(files)

In [ ]:
!ls ../public_tables/

In [ ]:
#load table entries
train_table='../public_tables/train.csv'
!ls -l $train_table

df=pd.read_csv(train_table,index_col=0)
df.index=df['filename']
df.shape

In [ ]:
#List the number of rows for each country, countries 0-4 are missing - the are in the country_test set
from collections import Counter
C=Counter(df['country'])
[(k,C[k]) for k in sorted(C)]

## train color encoder

Creates a KDtree in color space

In [ ]:
%%time
## Generate encoding tree
train_size,tree=train_encoder(files,max_images=500,tree_depth=8)

In [ ]:
%%time
## Encode all data using encoding tree
Enc_data=encoded_dataset(image_dir,df,tree,label_col='label')

In [ ]:
Enc_data.data.shape

In [ ]:

D=DataSplitter(Enc_data.data)

### Train and test

In [ ]:
# set parameters for XGBoost
param['max_depth']=2
param['num_round']=10
param

## Train on Urban only
Random split into train and test.

In [ ]:
%%time
urban=True
area= 'Urban' if urban else 'Rural'
selector=df['urban']==urban
subData=D.get_subset(selector)
subD=DataSplitter(subData)

train_selector=np.random.rand(subData.shape[0]) > 0.7
Train=subD.get_subset(train_selector)
Test=subD.get_subset(~train_selector)

param['num_round']=10
log10=simple_bootstrap(Train,Test,param,ensemble_size=30)
param['num_round']=100
log100=simple_bootstrap(Train,Test,param,ensemble_size=30)

styled_logs=[
    {   'log':log10,
        'style':['g:','g-'],
        'label':'10 iterations',
        'label_color':'g'
    },
    {   'log':log100,
        'style':['b:','b-'],
        'label':'100 iterations',
        'label_color':'b'
    }
]

_mean,_std=plot_scores(styled_logs,title=f'{area}Only: Split into train and test at random')

dump_urban = deepcopy({'styled_logs':styled_logs,
    'tree':tree,
    'mean':_mean,
    'std':_std})

## Train on Rural only
Random split into train and test.

In [ ]:
urban=False
area= 'Urban' if urban else 'Rural'
selector=df['urban']==urban
subData=D.get_subset(selector)
subD=DataSplitter(subData)

train_selector=np.random.rand(subData.shape[0]) > 0.7
Train=subD.get_subset(train_selector)
Test=subD.get_subset(~train_selector)

param['num_round']=10
log10=simple_bootstrap(Train,Test,param,ensemble_size=30)
param['num_round']=100
log100=simple_bootstrap(Train,Test,param,ensemble_size=30)

styled_logs=[
    {   'log':log10,
        'style':['y:','y-'],
        'label':'10 iterations',
        'label_color':'y'
    },
    {   'log':log100,
        'style':['m:','m-'],
        'label':'100 iterations',
        'label_color':'m'
    }
]

_mean,_std=plot_scores(styled_logs,title=f'{area}Only: Split into train and test at random')

pickle_file='data/Checkpoint.pk'
dump_rural=deepcopy({'styled_logs':styled_logs,
     'tree':tree,
     'mean':_mean,
     'std':_std})
pkl.dump([dump_urban, dump_rural],open(pickle_file,'wb'))
!ls -l $pickle_file